To run this project you will need the data:
- here: https://cloud.sagemath.com/projects/19d1c156-e6da-41a4-899e-11346c774f9f/files/locations.csv
- and here: https://cloud.sagemath.com/projects/19d1c156-e6da-41a4-899e-11346c774f9f/files/uber_4weeks.csv

In [4]:
import csv
from datetime import datetime, timedelta
import pprint

# Load in the location details info
locationsFile = open("locations.csv", "UR")
csvReader = csv.reader(locationsFile, delimiter=',', quotechar='"')
locationsDetails = {}
for row in csvReader:
    if csvReader.line_num > 1:
        locationsDetails[row[0]] = row[1:]
        
print locationsDetails

# Load in actual data for all the locations
# id,timestamp,surge_multiplier,expected_wait_time,locationID
csvFile = open("uber_4weeks.csv", 'Ur')
csvReader = csv.reader(csvFile, delimiter=',', quotechar='"')
locationsData = {}
for row in csvReader:
    if csvReader.line_num > 1:
        if row[4] not in locationsData:            
            locationsData[row[4]] = []
        # convert date    
        row[1] = datetime.strptime(row[1], '%Y-%m-%d %H:%M:%S')
        row[2] = float(row[2]) # surge price
        row[3] = float(row[3]) # estimated wait time
        
        new_row = row[1:4]
        new_row.extend([0])
        locationsData[row[4]].append(new_row)
        
#print locationsData["1"][0:5]

{'1': ['Washington,DC', 'U Street (U and 14th)', '38.916897', '-77.032115'], '37': ['Washington,DC', 'Adams Morgan', '38.921998', '-77.043291'], '3': ['Washington,DC', 'K Street', '38.902546', '-77.041764'], '5': ['Washington,DC', 'Paulding St. Navy Yard', '38.874025', '-76.995602'], '4': ['Washington,DC', 'George Washington University', '38.899582', '-77.04761']}


In [2]:
# This is a check on data integrity to see if there were any hiccups with data collection
# Data was collected at an interval of 15 seconds

# What's the avg difference to the next timepoint?
timeDiffHist = {}
timeDiffList = []
for (id, location) in locationsData.iteritems():
    #print id
    #print location
    #if id == "37":
    for i in range(1, len(location)):        
        time_diff = location[i][0] - location[i-1][0]
        time_diff_secs = time_diff.total_seconds()
        if time_diff_secs != 15:
            timeDiffList.append({"delay": time_diff_secs})

        if time_diff_secs not in timeDiffHist:
            timeDiffHist[time_diff_secs] = 0
        timeDiffHist[time_diff_secs] = timeDiffHist[time_diff_secs] + 1

pprint.pprint(timeDiffHist)
#print timeDiffList

{15.0: 780031,
 30.0: 6053,
 45.0: 1723,
 60.0: 614,
 75.0: 363,
 90.0: 201,
 105.0: 99,
 120.0: 83,
 135.0: 41,
 150.0: 45,
 165.0: 31,
 180.0: 13,
 195.0: 9,
 210.0: 5,
 225.0: 7,
 240.0: 2,
 255.0: 1,
 270.0: 3,
 285.0: 3,
 345.0: 6,
 360.0: 2,
 375.0: 1,
 420.0: 4,
 435.0: 3,
 450.0: 1,
 705.0: 1}


In [5]:
# Look at time hiccups across diff locations
timeDiffHist = {}
timeDiffList = []
for (id, location) in locationsData.iteritems():
    timeDiffHist = {}
    #print id
    print locationsDetails[id]
    #if id == "37":
    for i in range(1, len(location)):        
        time_diff = location[i][0] - location[i-1][0]
        time_diff_secs = time_diff.total_seconds()
        if time_diff_secs != 15:
            timeDiffList.append({"delay": time_diff_secs})
            
        if time_diff_secs not in timeDiffHist:
            timeDiffHist[time_diff_secs] = 0
        timeDiffHist[time_diff_secs] = timeDiffHist[time_diff_secs] + 1

    pprint.pprint(timeDiffHist)

# Indeed the hiccups do vary by location and so the hiccup is likely the result of an API call to that location that didn't return. 

['Washington,DC', 'U Street (U and 14th)', '38.916897', '-77.032115']
{15.0: 158253,
 30.0: 868,
 45.0: 177,
 60.0: 59,
 75.0: 34,
 90.0: 15,
 105.0: 8,
 120.0: 6,
 135.0: 3,
 150.0: 2,
 165.0: 3,
 345.0: 1,
 420.0: 2}
['Washington,DC', 'Paulding St. Navy Yard', '38.874025', '-76.995602']
{15.0: 146792,
 30.0: 2796,
 45.0: 1013,
 60.0: 389,
 75.0: 235,
 90.0: 137,
 105.0: 63,
 120.0: 58,
 135.0: 26,
 150.0: 35,
 165.0: 21,
 180.0: 9,
 195.0: 5,
 210.0: 4,
 225.0: 4,
 240.0: 2,
 255.0: 1,
 270.0: 2,
 285.0: 3,
 345.0: 1,
 375.0: 1,
 420.0: 1,
 435.0: 1,
 705.0: 1}
['Washington,DC', 'K Street', '38.902546', '-77.041764']
{15.0: 159484,
 30.0: 500,
 45.0: 115,
 60.0: 21,
 75.0: 20,
 90.0: 9,
 105.0: 4,
 120.0: 2,
 135.0: 3,
 150.0: 2,
 180.0: 1,
 225.0: 1,
 270.0: 1,
 345.0: 1,
 360.0: 1,
 435.0: 1}
['Washington,DC', 'Adams Morgan', '38.921998', '-77.043291']
{15.0: 156291,
 30.0: 1356,
 45.0: 305,
 60.0: 102,
 75.0: 52,
 90.0: 29,
 105.0: 16,
 120.0: 10,
 135.0: 4,
 150.0: 3,
 165.0: 6,


----
**How often do surge prices change?** 

In [6]:
import numpy as np

# For 1 day how many times has the surge price changed as each location
locationPriceChanges = {}
avg_price_change_pos = {}
avg_price_change_neg = {}
for (id, location) in locationsData.iteritems():
    print locationsDetails[id]
    for i in range(1, len(location)):   
        price_change = location[i][1] - location[i-1][1]
        if price_change != 0:
            if id not in locationPriceChanges:
                locationPriceChanges[id] = 0
            location[i][3] = round(100*price_change) / 100
            
            if id not in avg_price_change_pos:
                avg_price_change_pos[id] = []
                
            if id not in avg_price_change_neg:
                avg_price_change_neg[id] = []
            
            if price_change > 0:
                avg_price_change_pos[id].append(price_change)
            
            if price_change < 0:
                avg_price_change_neg[id].append(price_change)
                
            locationPriceChanges[id] = locationPriceChanges[id] + 1
    print "Num price changes at this location: "
    print " " + str(locationPriceChanges[id])
    print "Avg price hike at location: "
    print " " + str(np.mean(np.array(avg_price_change_pos[id])))
    print "Avg price reduction at location: "
    print " " + str(np.mean(np.array(avg_price_change_neg[id])))


['Washington,DC', 'U Street (U and 14th)', '38.916897', '-77.032115']
Num price changes at this location: 
 5510
Avg price hike at location: 
 0.394484660462
Avg price reduction at location: 
 -0.438597163664
['Washington,DC', 'Paulding St. Navy Yard', '38.874025', '-76.995602']
Num price changes at this location: 
 4604
Avg price hike at location: 
 0.357561983471
Avg price reduction at location: 
 -0.396108058608
['Washington,DC', 'K Street', '38.902546', '-77.041764']
Num price changes at this location: 
 4833
Avg price hike at location: 
 0.351010701546
Avg price reduction at location: 
 -0.383333333333
['Washington,DC', 'Adams Morgan', '38.921998', '-77.043291']
Num price changes at this location: 
 5510
Avg price hike at location: 
 0.394484660462
Avg price reduction at location: 
 -0.438597163664
['Washington,DC', 'George Washington University', '38.899582', '-77.04761']
Num price changes at this location: 
 4833
Avg price hike at location: 
 0.351010701546
Avg price reduction a

---
**From the below, take the pricechanges.csv and plug it into Tableau to plot a histogram of price changes by time or by magnitude**

In [7]:
# Output a .csv that I can pull into Tableau to aggregate by hour or magnitude

fileWriter = csv.writer(open("pricechanges.csv", "w+"),delimiter=",")
fileWriter.writerow(["location id", "datetime", "surge price", "estimated waiting time", "price change"])
priceChangeList = []
for (id, location) in locationsData.iteritems():   
    for i in range(1, len(location)):           
        if location[i][3] != 0:
            row = [id, location[i][0], location[i][1], location[i][2], location[i][3]]
            row[1] = row[1].isoformat()
            priceChangeList.append(row)
            fileWriter.writerow(row)

---
**Q: What's the average price change up or down?**

The general trend holds across the different locations, but the absolute magnitudes vary (see above for the calculation).

In [8]:
# An interesting result would be to count the number of price changes > 0 and < 0
num_positive_price_changes = 0
total_positive_price_changes = 0
num_negative_price_changes = 0
total_negative_price_changes = 0
for row in priceChangeList:
    if row[4] > 0:
        num_positive_price_changes = num_positive_price_changes + 1
        total_positive_price_changes = total_positive_price_changes + row[4]
    if row[4] < 0:
        num_negative_price_changes = num_negative_price_changes + 1
        total_negative_price_changes = total_negative_price_changes + row[4]
        
print num_positive_price_changes
print total_positive_price_changes
print total_positive_price_changes / num_positive_price_changes
print num_negative_price_changes
print total_negative_price_changes
print total_negative_price_changes / num_negative_price_changes

13268
4925.3
0.371216460657
12022
-4924.7
-0.409640658792


In [9]:
# Because of the data hiccups ... 
# Want to interpolate for missing data points (linear) in both the surge price and in the estimated wait times ...
td = timedelta(seconds=15)
locationsDataInterpolated = {}
for (id, location) in locationsData.iteritems():
    for i in range(0, len(location)-1):  
        # Copy over
        if id not in locationsDataInterpolated:
            locationsDataInterpolated[id] = []
        locationsDataInterpolated[id].append(location[i])
        
        # Interpolate
        time_diff = location[i+1][0] - location[i][0]
        time_diff_secs = time_diff.total_seconds()
        # If the time between two points is more than 15 seconds then we need to insert
        if time_diff_secs != 15:
            num_intervals = int(time_diff_secs / 15)
            for j in range(1, num_intervals):
                interpolated_price = (location[i+1][1] - location[i][1]) / (num_intervals)
                interpolated_estimated_wait_time = (location[i+1][2] - location[i][2]) / (num_intervals)
                interpolated_time = location[i][0] + j * td
                locationsDataInterpolated[id].append([interpolated_time, interpolated_price, interpolated_estimated_wait_time, 0])

# Sanity check 
timeDiffHist = {}
for (id, location) in locationsDataInterpolated.iteritems():
    #print id
    for i in range(1, len(location)):   
        #if id == 37:
        #print str(location[i][0].second) + " " + str(location[i][1])
        time_diff = location[i][0] - location[i-1][0]
        time_diff_secs = time_diff.total_seconds()
       
        if time_diff_secs not in timeDiffHist:
            timeDiffHist[time_diff_secs] = 0
        timeDiffHist[time_diff_secs] = timeDiffHist[time_diff_secs] + 1
pprint.pprint(timeDiffHist)


{15.0: 806390}


In [10]:
# Now do a lag analysis, waiting time lags price
# The output .csv is brought into Tableau to generate a nice graph. 
fileWriter = csv.writer(open("lag_analysis_ewt_lag.csv", "w+"),delimiter=",")
fileWriter.writerow(["location id", "lag_secs", "correlation"])

# offset the estimated wait time by shifting it "up" (which will line up surge price to correlate with estimated wait time at that delay)
# Run a pearson correlation and look for the delay that maximizes that correlation
import scipy
from scipy import stats
for (id, location) in locationsDataInterpolated.iteritems():
    print locationsDetails[id][1]
    price = []
    estimated_wait = []
    for i in range(1, len(location)):  
        price.append(location[i][1])
        estimated_wait.append(location[i][2])
    
    # For a delay from 0 to 15 minutes in 15 sec increments, estimated wait time lagged behind price
    for delay in range(0,61):
        estimated_wait_delay = scipy.array(estimated_wait[delay:])
       
        if delay > 0:
            price_delay = scipy.array(price[:-delay])
        else:
            price_delay = scipy.array(price[:])
              
        correlation = stats.pearsonr(price_delay, estimated_wait_delay)
        row = [locationsDetails[id][1], delay*15, correlation[0]]
        fileWriter.writerow(row)
        print row

U Street (U and 14th)
['U Street (U and 14th)', 0, 0.18359360332366265]
['U Street (U and 14th)', 15, 0.11042472972612528]
['U Street (U and 14th)', 30, 0.097119762197543952]
['U Street (U and 14th)', 45, 0.085856188157029251]
['U Street (U and 14th)', 60, 0.077145032976836614]
['U Street (U and 14th)', 75, 0.07151239137726817]
['U Street (U and 14th)', 90, 0.068465576279419821]
['U Street (U and 14th)', 105, 0.065600459682106718]
['U Street (U and 14th)', 120, 0.063499133694722695]
['U Street (U and 14th)', 135, 0.062022554601972001]
['U Street (U and 14th)', 150, 0.063183052518418187]
['U Street (U and 14th)', 165, 0.066110219495102859]
['U Street (U and 14th)', 180, 0.065565543946336222]
['U Street (U and 14th)', 195, 0.069782484345022752]
['U Street (U and 14th)', 210, 0.071897657340272855]
['U Street (U and 14th)', 225, 0.073741597754261687]
['U Street (U and 14th)', 240, 0.077559592308527811]
['U Street (U and 14th)', 255, 0.080558317657749762]
['U Street (U and 14th)', 270, 0.08

In [11]:
# Now do a lag analysis, price lags waiting time
fileWriter = csv.writer(open("lag_analysis_price_lag.csv", "w+"),delimiter=",")
fileWriter.writerow(["location id", "lag_secs", "correlation"])

# offset the estimated wait time by shifting it "up" (which will line up surge price to correlate with estimated wait time at that delay)
# Run a pearson correlation and look for the delay that maximizes that correlation
import scipy
from scipy import stats
for (id, location) in locationsDataInterpolated.iteritems():
    print locationsDetails[id][1]
    price = []
    estimated_wait = []
    for i in range(1, len(location)):  
        price.append(location[i][1])
        estimated_wait.append(location[i][2])
    
    # For a delay from 0 to 15 minutes in 15 sec increments, price lagged between estimated wait (maybe wait time predicts price in the future?)
    for delay in range(0,61):
        price_delay = scipy.array(price[delay:])
       
        if delay > 0:
            estimated_wait_delay = scipy.array(estimated_wait[:-delay])
        else:
            estimated_wait_delay = scipy.array(estimated_wait[:])
              
        correlation = stats.pearsonr(price_delay, estimated_wait_delay)
        row = [locationsDetails[id][1], delay*15, correlation[0]]
        fileWriter.writerow(row)
        # print row
 

U Street (U and 14th)
Adams Morgan
K Street
Paulding St. Navy Yard
George Washington University


In [12]:
import scipy
from scipy import stats

# Now I want to measure whether if a price surge in one neighborhood saps the capacity in an adjacent one leading to worse wait times there.
# Now do a lag analysis
fileWriter = csv.writer(open("lag_analysis_p1p5_e5.csv", "w+"),delimiter=",")
fileWriter.writerow(["location id1", "location id2", "lag_secs", "correlation"])

# And data for a specific lag that leads to highest correlation 
fileWriter2 = csv.writer(open("lag_analysis_p1p5_e5_135secs.csv", "w+"),delimiter=",")
fileWriter2.writerow(["lag_secs", "price diff", "estimated wait time"])

# The location IDs to test between
locID1 = "1"
locID2 = "5"
location1 = locationsDataInterpolated[locID1]
location2 = locationsDataInterpolated[locID2]

price = []
estimated_wait = []
for index, datum1 in enumerate(location1):
    #print index, datum1

    datum2 = location2[index]
    price_diff = round((datum1[1] - datum2[1]), 1)   # this is how much more the price surge is in location 1
    price.append(price_diff)  
    estimated_wait.append(datum2[2]) # the wait time in location 2
    
# For a delay from 0 to 15 minutes in 15 sec increments, price lagged between estimated wait (maybe wait time predicts price in the future?)
for delay in range(0,61):
    estimated_wait_delay = scipy.array(estimated_wait[delay:])

    if delay > 0:
        price_delay = scipy.array(price[:-delay])
    else:
        price_delay = scipy.array(price[:])
    
    correlation = stats.pearsonr(price_delay, estimated_wait_delay)
    row = [locationsDetails[locID1][1], locationsDetails[locID2][1], delay*15, correlation[0]]
    fileWriter.writerow(row)
    
    # Also output the raw data for the max correlation (determined by running this block and looking at the output)
    if delay *15 == 135:
        for j, d in enumerate(price_delay):
            fileWriter2.writerow([delay*15, price_delay[j], estimated_wait_delay[j]])
    print row

['U Street (U and 14th)', 'Paulding St. Navy Yard', 0, -0.13404169184852219]
['U Street (U and 14th)', 'Paulding St. Navy Yard', 15, -0.0098373686102935804]
['U Street (U and 14th)', 'Paulding St. Navy Yard', 30, 0.0066090996345388973]
['U Street (U and 14th)', 'Paulding St. Navy Yard', 45, 0.017286405306610745]
['U Street (U and 14th)', 'Paulding St. Navy Yard', 60, 0.025394515453695678]
['U Street (U and 14th)', 'Paulding St. Navy Yard', 75, 0.029094270241210706]
['U Street (U and 14th)', 'Paulding St. Navy Yard', 90, 0.033917631867609585]
['U Street (U and 14th)', 'Paulding St. Navy Yard', 105, 0.035829541622402261]
['U Street (U and 14th)', 'Paulding St. Navy Yard', 120, 0.037129342598173999]
['U Street (U and 14th)', 'Paulding St. Navy Yard', 135, 0.038168261547469655]
['U Street (U and 14th)', 'Paulding St. Navy Yard', 150, 0.037285538888954105]
['U Street (U and 14th)', 'Paulding St. Navy Yard', 165, 0.036661265853524902]
['U Street (U and 14th)', 'Paulding St. Navy Yard', 180, 